In [ ]:
# Relevant module imports and installs
!pip install pulp pandas brotli
import pandas as pd
import pulp as plp
import sys 
import os
from collections import defaultdict

In [ ]:
# Get the absolute path to the directory containing the Python file
module_path = os.path.abspath(os.path.join('..', '..'))

# Add the directory to sys.path
if module_path not in sys.path:
    sys.path.append(module_path)

# Import the data retrieval functions
from challenge_data import get_bootstrap_static_data, get_gameweek_data, update_with_gameweek_cost

# Get the actual GW4 data
bootstrap_data = get_bootstrap_static_data()
gw4_data = update_with_gameweek_cost(get_gameweek_data(4, bootstrap_data), '2024-25', 4)

In [ ]:
# Get the number of players and their list of ids
player_ids = gw4_data['ID'].tolist()
player_count = len(player_ids)

# Set up the problem
model = plp.LpProblem("fpl-gw4-challenge-hindsight", plp.LpMaximize)

# Define the decision variables
lineup = [
    plp.LpVariable(f"lineup_{i}", lowBound=0, upBound=1, cat="Integer")
    for i in player_ids
]

# Define captain variables
captain = [
    plp.LpVariable(f"captain_{i}", lowBound=0, upBound=1, cat="Integer")
    for i in player_ids
]

# Set the objective function (the number of points scored by the team, with captain's points doubled)
model += plp.lpSum([lineup[i] * gw4_data.loc[i, 'Points'] for i in range(player_count)]) + \
         plp.lpSum([captain[i] * gw4_data.loc[i, 'Points'] for i in range(player_count)])

# Constraints

# Total number of players = 5 (2 goalkeepers + 3 defenders)
model += plp.lpSum(lineup) == 5

# Exactly one captain
model += plp.lpSum(captain) == 1

# Captain must be in the lineup
for i in range(player_count):
    model += captain[i] <= lineup[i]

# Exactly 2 Goalkeeper
model += plp.lpSum([lineup[i] for i in range(player_count) if gw4_data.loc[i, 'Position'] == 'Goalkeeper']) == 2

# Exactly 3 Defenders
model += plp.lpSum([lineup[i] for i in range(player_count) if gw4_data.loc[i, 'Position'] == 'Defender']) == 3

# Budget constraint: Total cost must be less than or equal to 30m
model += plp.lpSum([lineup[i] * gw4_data.loc[i, 'Cost'] for i in range(player_count)]) <= 30

# Solve the problem
model.solve(plp.PULP_CBC_CMD(msg=False))

# Function to print players by position
def print_players_by_position(players_dict):
    total_points = 0
    total_cost = 0
    for position in ['Goalkeeper', 'Defender', 'Midfielder', 'Forward']:
        if position in players_dict:
            print(f"\n{position}:")
            for player in players_dict[position]:
                captain_str = " (C)" if player['Captain'] else ""
                points = player['Points'] * (2 if player['Captain'] else 1)
                print(f"  {player['Name']}{captain_str} - {player['Team']} - Cost: {player['Cost']}m - Points: {points}")
                total_points += points
                total_cost += player['Cost']
    print(f"\nTotal Points: {round(total_points, 2)}")
    print(f"Total Cost: {round(total_cost, 2)}m")

# Print the results
print("Status:", plp.LpStatus[model.status])
selected_players = defaultdict(list)
for i in range(player_count):
    if lineup[i].value() == 1:
        player = gw4_data.loc[i]
        selected_players[player['Position']].append({
            'Name': player['Name'],
            'Team': player['Team'],
            'Cost': player['Cost'],
            'Points': player['Points'],
            'Captain': captain[i].value() == 1
        })

print("\nOptimal Lineup:")
print_players_by_position(selected_players)